In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect

# Crear la cadena de conexión
username = 'usuario_remoto'
password = 'Cla-Ve-Se-Gu-Ra-77'
host = '192.168.1.21'
database = 'museo2'

# Crear la cadena de conexión
connection_string = f'mysql+pymysql://{username}:{password}@{host}/{database}'

# Crear el motor de conexión
engine = create_engine(connection_string)

# Crear el inspector
inspector = inspect(engine)

# Obtener los nombres de las tablas
table_names = inspector.get_table_names()
print(f"Tablas en la base de datos: {table_names}")

# Leer la tabla 'Artefacto'
query = 'SELECT * FROM Artefacto'
df = pd.read_sql(query, engine)
print(df)


Tablas en la base de datos: ['Artefacto', 'Artefacto_Exhibicion', 'Exhibicion', 'Museo', 'Museo_Exhibicion', 'Recorrido', 'Usuario']
   id                  descripcion    familia       imagen       nombre
0   1  Descripción del artefacto 1  Familia A  imagen1.jpg  Artefacto 1
1   2  Descripción del artefacto 2  Familia B  imagen2.jpg  Artefacto 2
2   3  Descripción del artefacto 3  Familia A  imagen3.jpg  Artefacto 3
3   4  Descripción del artefacto 4  Familia C  imagen4.jpg  Artefacto 4
4   5  Descripción del artefacto 5  Familia B  imagen5.jpg  Artefacto 5
